In [2]:
!pip install -q flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 6.0 MB/s eta 0:00:00


In [22]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('/content/X_train.csv')
X_test = pd.read_csv('/content/X_test.csv')
y_train = pd.read_csv('/content/y_train.csv')
y_test = pd.read_csv('/content/y_test.csv')

In [11]:
from flaml import AutoML
automl = AutoML()

parametrs = {
     'time_budget': 1800,
     'metric': 'roc_auc',
     'log_file_name': 'log.txt',
     'log_training_metric': True,
     'task': 'classification'
}

automl.fit(X_train=X_train, y_train=y_train['label'], **parametrs)

[flaml.automl.logger: 10-23 21:06:21] {1728} INFO - task = classification
[flaml.automl.logger: 10-23 21:06:21] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 10-23 21:06:21] {1838} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 10-23 21:06:21] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 10-23 21:06:21] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 10-23 21:06:26] {2393} INFO - Estimated sufficient time budget=49083s. Estimated necessary time budget=1136s.
[flaml.automl.logger: 10-23 21:06:26] {2442} INFO -  at 6.2s,	estimator lgbm's best error=0.1902,	best estimator lgbm's best error=0.1902
[flaml.automl.logger: 10-23 21:06:26] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-23 21:06:28] {2442} INFO -  at 8.0s,	estimator lgbm's best error=0.1902,	best estimator lgbm's best error=0.1902
[flaml.automl.logger: 10-

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 10-23 21:06:41] {2442} INFO -  at 20.7s,	estimator sgd's best error=0.4112,	best estimator lgbm's best error=0.1403
[flaml.automl.logger: 10-23 21:06:41] {2258} INFO - iteration 7, current learner lgbm
[flaml.automl.logger: 10-23 21:06:43] {2442} INFO -  at 22.9s,	estimator lgbm's best error=0.1403,	best estimator lgbm's best error=0.1403
[flaml.automl.logger: 10-23 21:06:43] {2258} INFO - iteration 8, current learner sgd
[flaml.automl.logger: 10-23 21:06:45] {2442} INFO -  at 25.1s,	estimator sgd's best error=0.4112,	best estimator lgbm's best error=0.1403
[flaml.automl.logger: 10-23 21:06:45] {2258} INFO - iteration 9, current learner lgbm
[flaml.automl.logger: 10-23 21:06:47] {2442} INFO -  at 27.1s,	estimator lgbm's best error=0.1403,	best estimator lgbm's best error=0.1403
[flaml.automl.logger: 10-23 21:06:47] {2258} INFO - iteration 10, current learner sgd
[flaml.automl.logger: 10-23 21:06:50] {2442} INFO -  at 29.9s,	estimator sgd's best error=0.4112,	best 

INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.automl.logger: 10-23 21:36:25] {2442} INFO -  at 1805.1s,	estimator lrl1's best error=0.3895,	best estimator lgbm's best error=0.1190
[flaml.automl.logger: 10-23 21:36:43] {2685} INFO - retrain lgbm for 17.8s
[flaml.automl.logger: 10-23 21:36:43] {2688} INFO - retrained model: LGBMClassifier(colsample_bytree=0.639343560735908,
               learning_rate=0.06768177253578582, max_bin=1023,
               min_child_samples=4, n_estimators=1, n_jobs=-1, num_leaves=10,
               reg_alpha=0.015893117884432133, reg_lambda=0.13929328197891047,
               verbose=-1)
[flaml.automl.logger: 10-23 21:36:43] {1985} INFO - fit succeeded
[flaml.automl.logger: 10-23 21:36:43] {1986} INFO - Time taken to find the best model: 1734.4693546295166


In [12]:
automl.best_estimator

'lgbm'

In [13]:
1 - automl.best_loss

0.8810269177840967

In [14]:
params = automl.best_config
params

{'n_estimators': 2152,
 'num_leaves': 10,
 'min_child_samples': 4,
 'learning_rate': 0.06768177253578582,
 'log_max_bin': 10,
 'colsample_bytree': 0.639343560735908,
 'reg_alpha': 0.015893117884432133,
 'reg_lambda': 0.13929328197891047}

In [15]:
import joblib

best_model = automl.model.estimator
joblib.dump(best_model, 'best_lgbm_model.pkl')

['best_lgbm_model.pkl']

In [46]:
loaded_model = joblib.load('/content/best_lgbm_model.pkl')

predictions = loaded_model.predict_proba(X_test[['dates_len', 'period_days',
       'values_mean', 'values_min', 'values_max', 'values_std',
       'values_median', 'mean_diff', 'Q3', 'IQR',
       'outliers', 'unique_months', 'unique_years', 'values_skew',
       'values_kurtosis', 'max_diff', 'min_diff', 'values_pct_10',
       'values_pct_90', 'fft_max', 'fft_mean', 'fft_std',
       'autocorr_lag1', 'autocorr_lag2', 'n_peaks']])

In [47]:
from sklearn.metrics import classification_report, roc_auc_score

print("ROC AUC Score:", roc_auc_score(y_test, predictions[:, 1]))

ROC AUC Score: 0.8626346507256905
